In [1]:
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from scipy import integrate
from scipy import interpolate
import math
from scipy import signal


In [27]:
from numpy import exp, array, random, dot


class NeuralNetwork():
    def __init__(self):
        # Seed the random number generator
        random.seed(1)

        # We model a single neuron, with 3 input connections and 1 output connection.
        # We assign random weights to a 3 x 1 matrix, with values in the range -1 to 1
        # and mean 0.
        self.weights = 2 * random.random((3, 1)) - 1

    # The Sigmoid function, which describes an S shaped curve.Normalises to between 0 and 1
    def sigmoid(self, x):
        return 1 / (1 + exp(-x))

    # The derivative/gradient of the Sigmoid function. How confident are we about each weight
    def sigmoid_derivative(self, x):
        return x * (1 - x)

    def train(self, training_in, training_out, iters):
        for iteration in range(iters):
            
            output = self.think(training_set_inputs)

            # Calculate the error
            error = training_set_outputs - output

            adjustment = dot(training_in.T, error * self.sigmoid_derivative(output))

            # Adjust the weights.
            self.weights += adjustment

    # The neural network thinks.
    def think(self, inputs):
        return self.sigmoid(dot(inputs, self.weights))



neural_network = NeuralNetwork()

print("Random starting synaptic weights: ")
print(neural_network.weights)

training_in = array([[0, 0, 1], 
                     [1, 1, 1], 
                     [1, 0, 1], 
                     [0, 1, 1]])

training_out = array([[0, 1, 1, 0]]).T

neural_network.train(training_in, training_out, 100)

print("New synaptic weights after training: ")
print(neural_network.weights)

# Test the neural network with a new situation.
print("Considering new situation [1, 0, 0] -> ?: ")
print(neural_network.think(array([1, 0, 0])))

Random starting synaptic weights: 
[[-0.16595599]
 [ 0.44064899]
 [-0.99977125]]
New synaptic weights after training: 
[[ 4.62128737]
 [-0.20338466]
 [-2.09325311]]
Considering new situation [1, 0, 0] -> ?: 
[0.99025576]
